In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from surprise import Dataset, Reader, accuracy, NormalPredictor, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD, BaselineOnly, SVDpp, NMF, SlopeOne, CoClustering
from surprise.accuracy import rmse
from surprise.prediction_algorithms import SVD, SVDpp, NMF, BaselineOnly, NormalPredictor
from IPython.core.display import HTML
from surprise.model_selection.split import train_test_split as surprise_train_test_split
from surprise.model_selection import GridSearchCV, cross_validate

%matplotlib inline

In [69]:
user_ratings_df = pd.read_csv("./Data/user_reviews_no_zero.csv", index_col=[0])

In [70]:
recipes_df = pd.read_csv("./Data/recipes_subcat_cleaned.csv", index_col=[0])

In [4]:
recipes_df.head()

,name,recipe_id,minutes,description,ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbs_pdv,recipe_type
0,arriba baked winter squash mexican style,137739,55,autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",51.5,0.0,13.0,0.0,2.0,0.0,4.0,"['vegetarian', 'low cal', 'low carb']"
1,a bit different breakfast pizza,31490,30,this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",173.4,18.0,0.0,17.0,22.0,35.0,1.0,"['low cal', 'low carb', 'sugar free', 'not veg..."
2,all in the kitchen chili,112140,130,this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",269.8,22.0,32.0,48.0,39.0,27.0,5.0,"['low cal', 'low carb', 'not vegetarian']"
3,alouette potatoes,59389,45,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",368.1,17.0,10.0,2.0,14.0,8.0,20.0,"['vegetarian', 'low cal']"
4,amish tomato ketchup for canning,44061,190,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",352.9,1.0,337.0,23.0,3.0,0.0,28.0,"['vegetarian', 'low cal']"


In [71]:
#Clean the ingredients column so each row is a list of strings
recipes_df["ingredients"] = recipes_df["ingredients"].str.replace("'", "")
recipes_df["ingredients"] = recipes_df["ingredients"].str.strip("[")
recipes_df["ingredients"] = recipes_df["ingredients"].str.strip("]")
recipes_df["ingredients"] = recipes_df["ingredients"].str.split(", ")

In [72]:
#Clean the recipe_type column so each row is a list of strings
recipes_df["recipe_type"] = recipes_df["recipe_type"].str.replace("'", "")
recipes_df["recipe_type"] = recipes_df["recipe_type"].str.strip("[")
recipes_df["recipe_type"] = recipes_df["recipe_type"].str.strip("]")
recipes_df["recipe_type"] = recipes_df["recipe_type"].str.split(", ")

## Setting up Surprise

In [161]:
user_ratings_df[["user_id", "recipe_id"]] = user_ratings_df[["user_id", "recipe_id"]].astype(int)

In [162]:
rating_surprise_df = user_ratings_df[["user_id", "recipe_id", "rating"]]

In [163]:
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(rating_surprise_df, reader)

In [164]:
trainset_full = surprise_data.build_full_trainset()

In [165]:
best_model = SVD(n_factors = 3,  n_epochs= 35, lr_all = .0025)
best_model.fit(trainset_full)

In [157]:
rating_surprise_df.isnull().sum()

user_id      0
recipe_id    0
rating       0
dtype: int64

In [166]:
best_model.predict(152118, 127155)

Prediction(uid=152118, iid=127155, r_ui=None, est=4.876801746492294, details={'was_impossible': False})

In [167]:
best_model.predict(152118, 79222)

Prediction(uid=152118, iid=79222, r_ui=None, est=4.794776004996293, details={'was_impossible': False})

In [168]:
## Subset data frame to show reviewers the products they have rated 

prior_ratings_df = pd.DataFrame(user_ratings_df.set_index("user_id"))
prior_ratings_df.drop(columns=["date", "rating", "review"], inplace=True)
prior_ratings_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 537267 entries, 56680 to 1122988
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   recipe_id  537267 non-null  int64
dtypes: int64(1)
memory usage: 8.2 MB


In [61]:
user_ratings_df.head(20)

,user_id,recipe_id,date,rating,review
10,56680,79222,2006-11-11,5.0,"Oh, This was wonderful! Had a soup and salad ..."
11,183565,79222,2006-02-13,5.0,Wow! My family loves this recipe and it is a ...
12,101823,79222,2006-03-21,5.0,Excellent chowder. This was the perfect warm-...
13,446143,79222,2008-02-01,4.0,"Oh, how wonderful! I doubled the crab, and ad..."
14,226989,79222,2008-03-07,4.0,DH and I enjoyed this. However I used it only ...
15,868654,79222,2008-10-07,5.0,Along with the onions we added in a square of ...
16,302867,79222,2010-01-05,5.0,Delish! Part of my New Years resolution.... ...
17,930021,79222,2010-06-25,5.0,My favorite chowder ever. I may try it with ot...
18,241697,79222,2010-07-10,5.0,Really great recipe!! Although I did not have ...
20,158966,79222,2010-10-20,5.0,10/18/10 Update. I have to rave again about th...


## Creating Recommender Function 

In [56]:
recipes_df.head(1)

,name,recipe_id,minutes,description,ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbs_pdv,recipe_type
0,arriba baked winter squash mexican style,137739,55,autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",51.5,0.0,13.0,0.0,2.0,0.0,4.0,"[vegetarian, low cal, low carb]"


In [87]:
not_reviewed = recipes_df.copy()
not_reviewed = not_reviewed[not_reviewed.recipe_id.isin(reviewed) == False]
not_reviewed.reset_index(inplace=True)

NameError: name 'reviewed' is not defined

In [101]:
recipes_df["set_ingredients"] = recipes_df["ingredients"].apply(lambda x: set(x))


In [102]:
type(recipes_df["set_ingredients"][0])

set

In [103]:
not_reviewed = recipes_df.copy()

In [104]:
not_reviewed = not_reviewed[not_reviewed.recipe_id.isin(reviewed) == False]
not_reviewed.reset_index(inplace=True)
    

In [140]:
user = 183565
ingredient_request = "onions"
sub_cat_request = "vegetarian"

In [138]:
reviewed = list(prior_ratings_df.loc[user,"recipe_id"])

KeyError: 183565

In [130]:
recipes_w_ingredient = []
for index, row in recipes_df.iterrows():
    if ingredient_request in row["set_ingredients"]:
        recipes_w_ingredient.append(row["recipe_id"])
 

In [141]:
recipes_w_ingredient_subcat = []
for index, row in recipes_df.iterrows():
    if ingredient_request in row["set_ingredients"] and sub_cat_request in row["recipe_type"]:
        recipes_w_ingredient_subcat.append(row["recipe_id"])

In [142]:
len(recipes_w_ingredient_subcat)

3181

In [131]:
ingredient_recipes = not_reviewed.copy()
#loc, iloc statement (conditional with loc where value is in the list)
#ingredient_recipes = 
ingredient_recipes = ingredient_recipes[ingredient_recipes.recipe_id.isin(recipes_w_ingredient)] #== True]
ingredient_recipes.reset_index(inplace=True)
    

In [132]:
ingredient_recipes

,level_0,index,name,recipe_id,minutes,description,ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbs_pdv,recipe_type,set_ingredients
0,64,64,mr grant you took half veal prince orloff,43164,120,i couldn't resist! named in honor of one of my all time favorite,"[veal roast, butter, oil, carrots, onions, parsley sprigs, bay leaf, thyme, salt, pepper, bacon]",510.6,50.0,10.0,21.0,90.0,70.0,1.0,"[low carb, not vegetarian]","{butter, parsley sprigs, salt, veal roast, oil, bay leaf, bacon, onions, pepper, carrots, thyme}"
1,86,86,smoked salmon cracker spread,99024,65,"from the land of lakes cookbook via a friend who knows how i like to play. great as an appetizer spread, rolled in a tortilla, or in sandwiches. lots of possibilities for innovation here! cook time is chilling time.","[cream cheese, salmon, salt, lemon juice, onions, liquid smoke]",132.4,14.0,0.0,8.0,21.0,26.0,0.0,"[low cal, low carb, sugar free, not vegetarian]","{cream cheese, salt, lemon juice, salmon, onions, liquid smoke}"
2,109,109,zupa ze swiezych grzybow polish mushroom soup,105069,40,posted in response to a request. from my grandmother's recipe files. it's rich and delicious.,"[wild mushrooms, onions, butter, lemon, juice of, water, beef broth, sour cream, flour, salt and pepper, fresh dill]",176.6,19.0,11.0,26.0,12.0,37.0,3.0,"[low cal, low carb, not vegetarian]","{butter, juice of, salt and pepper, wild mushrooms, fresh dill, lemon, water, beef broth, sour cream, flour, onions}"
3,134,134,ara s potato oup,69190,135,"i made this recipe myself when i was 13. i was on here trying to find a recipe for potato soup because i liked my mom's so much. when i was looking, i couldn't find a simple recipe for it that looked good, so i decided to make my own. now, my soup tastes even better than my mom's and i always make it for family.","[cream of chicken soup, cream of celery soup, potatoes, onions, celery, carrots, water, butter, milk, bouillon cubes, potato flakes]",346.7,21.0,23.0,53.0,17.0,35.0,16.0,"[low cal, not vegetarian]","{butter, bouillon cubes, water, cream of celery soup, celery, milk, potato flakes, cream of chicken soup, onions, carrots, potatoes}"
4,136,136,lplermagrone,522861,50,"älplermagronethe name doesn't translate perfectly into english. magrone is of course macaroni, but älpler doesn't only refer to the mountains and the people who live there, but more specifically to the men who would go up into the high pastures with the cows every summer. they had plenty of milk and cheese available to make this rich and delicious dish. this is traditionally serverd with fried onions on top and apple slices or sauce on the side. the cooking time is approximate. you can saute your onions while cooking the macaroni. a straight tubular pasta is what is commonly used penne, mostaccioli or rigatoni, would all work well.","[milk, salt, macaroni, cheese, fresh coarse ground black pepper, butter, onions, garlic cloves]",1003.8,72.0,21.0,103.0,69.0,143.0,37.0,[vegetarian],"{cheese, butter, fresh coarse ground black pepper, salt, milk, garlic cloves, onions, macaroni}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9867,231581,231590,zuppa du jour,326419,15,"i received an email from the www.sparkrecipes.com website this morning with this recipe featured. here is what the chef says: fantastic soup, try with other veggies that you like! note: the only change that i am going to try is to use jennie-o lean turkey bacon.","[bacon, onions, garlic cloves, celery, carrot, mushroom, red pepper flakes, salt, pepper, oregano, basil, low-sodium low-fat chicken broth, stewed tomatoes, orzo pasta]",154.9,12.0,30.0,16.0,9.0,13.0,5.0,"[low cal, low carb, not vegetarian]","{stewed tomatoes, oregano, orzo pasta, mushroom, carrot, red pepper flakes, low-sodium low-fat chicken broth, salt, celery, bacon, garlic cloves, basil, onions, pepper}"
9868,231613,231622,zwiebelbrotchen onion rolls,424875,210,"from das grosse buch vom backe

In [133]:
ingredient_recipes["predicted_rating"] = ingredient_recipes["recipe_id"].apply(lambda x: best_model.predict(user, x).est)
ingredient_recipes.sort_values(by="predicted_rating", ascending=False, inplace=True)
ingredient_recipes = ingredient_recipes[["name", "minutes", "description", "ingredients", "recipe_type", "predicted_rating"]] 

In [134]:
ingredient_recipes["predicted_rating"].value_counts()

4.888764    9872
Name: predicted_rating, dtype: int64

In [147]:
predictions = best_model.predict(183565, 99024)

In [148]:
predictions

Prediction(uid=183565, iid=99024, r_ui=None, est=4.729493901542436, details={'was_impossible': False})

In [124]:
user1 = user_ratings_df[user_ratings_df["user_id"] == "827374"]


In [127]:
user1["rating"].mode()

0    4.0
dtype: float64

In [126]:
user_ratings_df["rating"].mean()

4.729493901542436

In [181]:
def recommended_recipes():
    #set up to not truncate display
    pd.set_option("display.max_colwidth", None)
    
    user = int(input("user_id: "))
    ingredient_request = input("What ingredients do you need to use?  ")
    #n_ingredients = int(input("How many ingredients do you want to match? "))
    sub_cat_request = input("Recipe type? ")
    
    # Set up a list for each user of recipes alredy reviewed 
    reviewed = list(prior_ratings_df.loc[user,"recipe_id"])
    
    # Create a data frame that does not include recipes already reviewed 
    not_reviewed = recipes_df.copy()
    not_reviewed = not_reviewed[not_reviewed.recipe_id.isin(reviewed) == False]
    not_reviewed.reset_index(inplace=True)
    
    #Create a list from not_reviewed that only includes recipe with the requested ingredient
    #recipes_w_ingredient = []
    #for index, row in recipes_df.iterrows():
        #if ingredient_request in row["set_ingredients"]:
            #recipes_w_ingredient.append(row["recipe_id"])
            
            
    #subcategory added to function
    recipes_w_ingredient_subcat = []
    for index, row in recipes_df.iterrows():
        if ingredient_request in row["set_ingredients"] and sub_cat_request in row["recipe_type"]:
            recipes_w_ingredient_subcat.append(row["recipe_id"])
    
    ingredient_recipes = not_reviewed.copy()
    #loc, iloc statement (conditional with loc where value is in the list
    ingredient_recipes = ingredient_recipes[ingredient_recipes.recipe_id.isin(recipes_w_ingredient_subcat)]
    ingredient_recipes.reset_index(inplace=True)
    
    #not_reviewed = not_reviewed[not_reviewed["ingredients"].isin(ingredient_request) == True]
    
    ingredient_recipes["predicted_rating"] = ingredient_recipes["recipe_id"].apply(lambda x: best_model.predict(int(user), x).est)
    ingredient_recipes.sort_values(by="predicted_rating", ascending=False, inplace=True)
    ingredient_recipes = ingredient_recipes[["name", "minutes", "description", "ingredients", "recipe_type", "predicted_rating"]] 
    
    return ingredient_recipes.head(10)
    
    

In [186]:
recommended_recipes()

user_id: 827374
What ingredients do you need to use?  cheese
Recipe type? low carb


,name,minutes,description,ingredients,recipe_type,predicted_rating
139,beer burgers with beer braised onions,60,an excellent burger and the onions piled on top of the burger are great. this recipe is one of my dh's favorites.,"[butter, onion, beer, sugar, salt, ground chuck, tabasco sauce, worcestershire sauce, salt and pepper, hamburger buns, cheese]","[low carb, not vegetarian]",4.586301
1160,slow cooker pork chili verde,430,this is a great recipe to throw in the crock pot before work with very little prep work.,"[olive oil, onion, garlic cloves, boneless pork shoulder, green enchilada sauce, jalapeno peppers, flour tortillas, cheese, sour cream, salsa]","[low carb, not vegetarian]",4.560646
847,magic omelette,10,"this isn't so much a recipe as a tip. most of us make omelettes the same way: wait until the eggs are nearly cooked, then add cheese, vegetables, meat, etc., counting on the heat of the eggs to heat them, and in doing so unnecessarily overcooking the eggs. if, instead, you cover your egg pan with a pot lid, you can add the omelette filling from the outset and it cooks with the eggs, so your omelette's already done at the point where you'd normally add the filling. and get this - you never have to touch a utensil! try it and i bet you'll never cook an omelette without a lid again.","[butter, eggs, water, cheese, ham, chives, salt, pepper]","[low cal, low carb, not vegetarian]",4.560097
468,crustless swiss chard quiche,60,"i have been working on perfecting the perfect crustless quiche recipe for years. i always use spinach, but i bought some rainbow swiss chard at the local farmer's market today, and i thought i'd give it a try in quiche. i used rainbow chard (pink/yellow/white/orange stems---- beautiful), but you can use any chard (or try substituting other greens) i do notice that fresh greens give a much better taste than frozen greens though!","[olive oil, sweet onion, swiss chard, cheese, eggs, skim milk, salt, pepper]","[vegetarian, low cal, low carb]",4.559858
1218,steak or chicken fajitas,20,these are so quick and tasty. you can use strips of boneless chicken breast instead of the beef. feel free to double the marinade. the flavor is so great for such a short marination time. you can can do them up ahead of time and have the meat sit longer if you wish.,"[top sirloin steak, olive oil, lime juice, garlic clove, chili powder, cumin, hot pepper flakes, black pepper, salt, flour tortillas, onion, sweet peppers, salsa, sour cream, cheese, chopped tomato]","[low cal, low carb, not vegetarian]",4.544741
647,garlic grilled cheese,8,ahhh... gaaahlic! who doesn't love it? i was making a variation of the potato chip chicken recipe and melted a bit too much margarine. so i stuck it in the fridge and made this grilled cheese the next day. i'll never make plain grilled cheese again!,"[bread, cheese, margarine, garlic salt]","[vegetarian, low carb]",4.542237
1004,pizza lunchable oamc,30,"this recipe is in response to my son's desperate pleas for those lunchables that are unhealthy, expensive, and full of ingredients i can't pronounce...lol. since i refuse to buy them, i came up with this idea instead to compromise with him.","[pizza dough, pizza sauce, cheese, ziploc bags]","[vegetarian, low cal, low carb]",4.538666
1320,tomato and cheese quesadilla,10,"a very easy to make lunch. if following a gluten free diet ensure gluten-free corn tortillas are used. you can make these in your frypan or flat sandwich toaster (the type that does not cut your bread). you can also add cooked shredded chicken, ham, avocado,onion, capsicums/peppers, herbs, salsa, chilli sauce-basically any ingredients that you would like in a toasted sandwich.","[corn tortillas, cheese, tomatoes]","[vegetarian, low cal, low carb]",4.536997
1122,scalloped turnips,25,"mrs. carl albert submitted this in a 1971 cookbook called favorite recipes of our first ladies. her husband was speaker of the house and she said this is one of his favorite recipes because ""he 